# 1: Histogram generator

The following cell generates an interactive plot where you can change the mean value of the data, the standard deviation (stdev), the skew and number of points (npts) of the data set by moving the sliders. The right hand plot shows part of the random sample of data that is used to generate the distribution.

In [1]:
from ipywidgets import interactive
from figure_functions import *
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion()

iplot = interactive(plotpdf,mean=(-5.0,5.01,1.0),stdev=(0.1,2.1,0.2),skew=(-0.9,0.92,0.1),npts=(250,5001,250));
output = iplot.children[-1];
output.layout.height = '350px'
iplot

interactive(children=(FloatSlider(value=0.0, description='mean', max=5.01, min=-5.0, step=1.0), FloatSlider(va…

# 2: effect of outliers on quartile values, and on the mean +/- std

You can see the effect of outliers on the quartiles, and on the mean $\pm$ standard deviation by running the cell below, and moving the sliders in the figure produced below.  
The slider *npts* controls the number of data points generated, and *pc_anom* controls the percentage of "anomaly" points.

In [2]:
from ipywidgets import interactive
from figure_functions import *
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion()

iplot = interactive(plotquantiles,mean=(-3.0,3.01,1.0),stdev=(0.1,2.1,0.2),npts=(250,5001,250),pc_anomalies=(0,10.5,1));
output = iplot.children[-1];
output.layout.height = '350px'
iplot

interactive(children=(FloatSlider(value=0.0, description='mean', max=3.01, min=-3.0, step=1.0), FloatSlider(va…